In [ ]:
import os
import numpy as np
from IPython.core.debugger import set_trace

In [ ]:
sub2vec_dir = "sub2vec"

save_dir = "features"
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
    
input_trees = "incontrege_subtrees"
num_iters = 5

In [ ]:
all_probabilities = np.load("all_beam_probabilities.npy", allow_pickle=True)
print(all_probabilities.shape)

In [ ]:
for ite in range(num_iters):
    print("python {}/src/main.py --input {} --property n --output {}/incontrege_set_{} --d 15 --windowSize 5 --model dbon"\
              .format(sub2vec_dir, input_trees, save_dir, ite))
    if not os.path.exists("{}/incontrege_set_{}".format(save_dir, ite)):
        os.system("python {}/src/main.py --input {} --property n --output {}/incontrege_set_{} --d 15 --windowSize 5 --model dbon"\
              .format(sub2vec_dir, input_trees, save_dir, ite))    
    graph_emb_file = "{}/incontrege_set_{}".format(save_dir, ite)
    graph_emb_file = open(graph_emb_file,"r").readlines()
    graph_emb_file = graph_emb_file[1:]
    emb_len = 15
    final_embeddings = []
    for i in range(5176):
        final_embeddings.append([0.0]*emb_len)
    
    for line in graph_emb_file:
        line = line.strip().split("\t")
        if not line[0].startswith("subGraph"):
            continue
            
        wordnum, rank = line[0][8:].split("_")
        wordnum = int(wordnum)
        rank = int(rank)
        
        embs = line[1:]
        for i in range(emb_len):
            final_embeddings[wordnum][i] += (float(embs[i]) * all_probabilities[wordnum][rank])

    print(len(final_embeddings))
    print(len(final_embeddings[0]))

    np.save("{}/incontrege_set_{}.npy".format(save_dir, ite),final_embeddings)